In [1]:
# cnn practice
import torch
from d2l import torch as d2l
from torch.nn import functional as F
from torch import nn

In [2]:
# batch size and data iter
lr, num_epochs, batch_size = 0.05, 10, 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=96)

In [3]:
# model: Residual blk
class Residual(nn.Module):
    def __init__(self, input_channels, output_channels, use_1x1conv = False, strides = 1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, output_channels, 
                               kernel_size = 3, padding = 1, stride = strides)
        self.conv2 = nn.Conv2d(output_channels, output_channels, 
                               kernel_size = 3, padding = 1)

        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, output_channels, 
                                   kernel_size = 1, stride = strides)
        else:
            self.conv3 = None

        self.bn1 = nn.BatchNorm2d(output_channels)
        self.bn2 = nn.BatchNorm2d(output_channels)

    def forward(self, X):
        Y = self.bn2(self.conv2(F.relu(self.bn1(self.conv1(X)))))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

In [4]:
# test Residual blk
blk = Residual(3, 4, True)
X = torch.randn(3, 3, 6, 6)
blk(X).shape

torch.Size([3, 4, 6, 6])

In [5]:
# build resNet
b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride = 2, padding = 3),
                   nn.BatchNorm2d(64),
                   nn.ReLU(),
                   nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1))

def resnet_blk(input_channels, output_channels, number_blk, first_blk = False):
    blk = []
    for i in range(number_blk):
        if i == 0 and not first_blk:
            blk.append(Residual(input_channels, output_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(output_channels, output_channels))
    return blk

b2 = nn.Sequential(*resnet_blk(64, 64, 2, True))
b3 = nn.Sequential(*resnet_blk(64, 128, 2))
b4 = nn.Sequential(*resnet_blk(128, 256, 2))
b5 = nn.Sequential(*resnet_blk(256, 512, 2))

In [6]:
# build nets
net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

In [7]:
# test nets
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "output \t", X.shape)

Sequential output 	 torch.Size([1, 64, 56, 56])
Sequential output 	 torch.Size([1, 64, 56, 56])
Sequential output 	 torch.Size([1, 128, 28, 28])
Sequential output 	 torch.Size([1, 256, 14, 14])
Sequential output 	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output 	 torch.Size([1, 512, 1, 1])
Flatten output 	 torch.Size([1, 512])
Linear output 	 torch.Size([1, 10])


In [ ]:
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

# too slow in mac, so no output